In [ ]:
import numpy as np
import heapq
import matplotlib.pyplot as plt
import rasterio
import math

# Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c  # Distance in kilometers

# Convert latitude and longitude to map indices
def latlon_to_index(lat, lon, lat_min, lon_min, lat_res, lon_res):
    x = int((lat - lat_min) / lat_res)
    y = int((lon - lon_min) / lon_res)
    return (4050 - x, y)

def index_to_latlon(x, y, lat_min, lon_min, lat_res, lon_res):
    lat = lat_min + (4050 - x) * lat_res
    lon = lon_min + y * lon_res
    return (lat, lon)

# Calculate effective speed based on wind
def effective_speed(ship_speed, wind_speed, wind_angle, dx, dy):
    relative_angle = math.atan2(dy, dx) - wind_angle
    wind_effect = wind_speed * math.cos(relative_angle)
    return max(0, ship_speed + wind_effect)

# Check if a move is valid
def valid_move(x, y, binary_map):
    return 0 <= x < binary_map.shape[0] and 0 <= y < binary_map.shape[1] and binary_map[x, y] == 0

# A* Algorithm with wind effects
def a_star(start, goal, binary_map, wind_speed_map, wind_angle_map, ship_speed):
    open_list = []
    heapq.heappush(open_list, (0, 0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: 0}

    while open_list:
        _, current_g, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path, g_score[goal]

        neighbors = [
            (current[0] - 1, current[1]), (current[0] + 1, current[1]),
            (current[0], current[1] - 1), (current[0], current[1] + 1)
        ]

        for neighbor in neighbors:
            if valid_move(neighbor[0], neighbor[1], binary_map):
                lat1, lon1 = index_to_latlon(current[0], current[1], lat_min, lon_min, lat_res, lon_res)
                lat2, lon2 = index_to_latlon(neighbor[0], neighbor[1], lat_min, lon_min, lat_res, lon_res)

                distance = haversine(lat1, lon1, lat2, lon2)
                wind_speed = wind_speed_map[neighbor[0], neighbor[1]]
                wind_angle = wind_angle_map[neighbor[0], neighbor[1]]
                dx, dy = neighbor[0] - current[0], neighbor[1] - current[1]
                eff_speed = effective_speed(ship_speed, wind_speed, wind_angle, dx, dy)
                time_cost = distance / eff_speed if eff_speed > 0 else float('inf')

                tentative_g_score = current_g + time_cost

                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score
                    heapq.heappush(open_list, (f_score[neighbor], tentative_g_score, neighbor))

    return None, float('inf')

# Generate initial population
def generate_initial_population(a_star_path, binary_map, population_size=10):
    population = []
    for _ in range(population_size):
        new_path = a_star_path[:]
        for i in range(1, len(a_star_path) - 1):
            x, y = new_path[i]
            dx, dy = np.random.choice([-1, 0, 1]), np.random.choice([-1, 0, 1])
            new_x, new_y = x + dx, y + dy
            if valid_move(new_x, new_y, binary_map):
                new_path[i] = (new_x, new_y)
        population.append(new_path)
    return population

# Fitness function
def fitness_function(path, binary_map, wind_speed_map, wind_angle_map, ship_speed):
    total_time = 0
    for i in range(len(path) - 1):
        x1, y1 = path[i]
        x2, y2 = path[i + 1]
        
        lat1, lon1 = index_to_latlon(x1, y1, lat_min, lon_min, lat_res, lon_res)
        lat2, lon2 = index_to_latlon(x2, y2, lat_min, lon_min, lat_res, lon_res)
        
        distance = haversine(lat1, lon1, lat2, lon2)
        wind_speed = wind_speed_map[x2, y2]
        wind_angle = wind_angle_map[x2, y2]
        dx, dy = x2 - x1, y2 - y1
        eff_speed = effective_speed(ship_speed, wind_speed, wind_angle, dx, dy)
        
        if eff_speed > 0:
            total_time += distance / eff_speed
        else:
            return float('inf')
    
    return total_time

# Selection
def tournament_selection(population, fitnesses, k=3):
    selected = np.random.choice(range(len(population)), k, replace=False)
    best = min(selected, key=lambda idx: fitnesses[idx])
    return population[best]

# Crossover
def crossover(parent1, parent2):
    cut = np.random.randint(1, len(parent1) - 1)
    return parent1[:cut] + parent2[cut:]

# Mutation
def mutate(path, binary_map, mutation_rate=0.1):
    new_path = path[:]
    for i in range(1, len(path) - 1):
        if np.random.rand() < mutation_rate:
            x, y = new_path[i]
            dx, dy = np.random.choice([-1, 0, 1]), np.random.choice([-1, 0, 1])
            new_x, new_y = x + dx, y + dy
            if valid_move(new_x, new_y, binary_map):
                new_path[i] = (new_x, new_y)
    return new_path

# Genetic Algorithm
def genetic_algorithm(start, goal, binary_map, wind_speed_map, wind_angle_map, ship_speed, 
                      a_star_path, population_size=10, generations=50, mutation_rate=0.1):
    population = generate_initial_population(a_star_path, binary_map, population_size)
    best_path, best_fitness = None, float('inf')
    
    for generation in range(generations):
        fitnesses = [fitness_function(path, binary_map, wind_speed_map, wind_angle_map, ship_speed) for path in population]
        
        # Update best path
        for i, fitness in enumerate(fitnesses):
            if fitness < best_fitness:
                best_fitness = fitness
                best_path = population[i]
        
        # New generation
        new_population = []
        for _ in range(population_size // 2):
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child1 = mutate(crossover(parent1, parent2), binary_map, mutation_rate)
            child2 = mutate(crossover(parent2, parent1), binary_map, mutation_rate)
            new_population.extend([child1, child2])
        
        population = new_population
    
    return best_path, best_fitness

# Main
binary_file = "indian_ocean_binary.tif"
with rasterio.open(binary_file) as src:
    binary_map = src.read(1)

lat_min, lat_max = -60, 30
lon_min, lon_max = 20, 120
lat_res = (lat_max - lat_min) / binary_map.shape[0]
lon_res = (lon_max - lon_min) / binary_map.shape[1]

# Generate wind data
wind_speed_map = np.random.uniform(0, 20, binary_map.shape)
wind_angle_map = np.random.uniform(0, 2 * np.pi, binary_map.shape)

# Define start and goal
start = latlon_to_index(18.5, 72.5, lat_min, lon_min, lat_res, lon_res)  # Mumbai
goal = latlon_to_index(11, 90, lat_min, lon_min, lat_res, lon_res)   # Antarctica region
ship_speed = 50

# Run A* algorithm
a_star_path, _ = a_star(start, goal, binary_map, wind_speed_map, wind_angle_map, ship_speed)

# Run Genetic Algorithm
ga_path, ga_time = genetic_algorithm(start, goal, binary_map, wind_speed_map, wind_angle_map, ship_speed, a_star_path)

if ga_path:
    print(len(ga_path))
    total_ga_time = fitness_function(ga_path, binary_map, wind_speed_map, wind_angle_map, ship_speed)
    print(f"Overall time to reach the goal using GA: {total_ga_time:.2f} hours")
else:
    print("No path found using Genetic Algorithm.")

# Plot result
plt.imshow(binary_map, cmap='gray', origin='upper')
if ga_path:
    path_x, path_y = zip(*ga_path)
    plt.plot(path_y, path_x, color='red', linewidth=2, label='GA Path')
plt.scatter([start[1], goal[1]], [start[0], goal[0]], c=['green', 'blue'], label='Start/Goal')
plt.legend()
plt.title("Genetic Algorithm Path")
plt.show()
